In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

In [4]:
train=pd.read_csv('D:/abhipraja/codes/dataset/titanic/train.csv')
test=pd.read_csv('D:/abhipraja/codes/dataset/titanic/test.csv')

In [5]:
train.info(),test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

(None, None)

In [6]:
ytrain=train['Survived']

In [7]:
train.drop(labels='Survived',axis=1,inplace=True)

In [8]:
train.shape,test.shape

((891, 11), (418, 11))

In [9]:
trainTest=train.append(test)

C:\Users\abhipraja\AppData\Local\Temp\ipykernel_11328\2294878807.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trainTest=train.append(test)


In [11]:
columnsToDrop=['Name','Age','SibSp','Ticket','Cabin','Parch','Embarked']
trainTest.drop(labels=columnsToDrop,axis=1,inplace=True)

In [12]:
trainTestDummies=pd.get_dummies(trainTest,columns=['Sex'])

In [13]:
trainTestDummies.shape

(1309, 5)

In [15]:
xtrain=trainTestDummies.values[0:891]
xtest=trainTestDummies.values[891:]

In [16]:
xtrain.shape,xtest.shape

((891, 5), (418, 5))

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
xtrainScale=scaler.fit_transform(xtrain)
xtestScale=scaler.transform(xtest)

In [19]:
from sklearn.model_selection import train_test_split
xtrainSub,xvalidationSub,ytrainSub,yvalidationSub=train_test_split(xtrainScale,ytrain,random_state=0)

In [20]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,auc

In [21]:
learningRates=[.05,.1,.25,.5,.75,1]
for learningRate in learningRates:
    gb=GradientBoostingClassifier(n_estimators=20,learning_rate=learningRate,max_features=2,max_depth=2,random_state=0)
    gb.fit(xtrainSub,ytrainSub)
    print('learing rates:',learningRate)
    print('accuracy score(training):{0:.3f}'.format(gb.score(xtrainSub,ytrainSub)))
    print('accuracy score(validaton):{0:.3f}'.format(gb.score(xvalidationSub,yvalidationSub)))
    print()


learing rates: 0.05
accuracy score(training):0.778
accuracy score(validaton):0.798

learing rates: 0.1
accuracy score(training):0.804
accuracy score(validaton):0.816

learing rates: 0.25
accuracy score(training):0.819
accuracy score(validaton):0.798

learing rates: 0.5
accuracy score(training):0.838
accuracy score(validaton):0.812

learing rates: 0.75
accuracy score(training):0.844
accuracy score(validaton):0.821

learing rates: 1
accuracy score(training):0.852
accuracy score(validaton):0.807



In [22]:
gb=GradientBoostingClassifier(n_estimators=20,learning_rate=.5,max_features=2,max_depth=2,random_state=0)
gb.fit(xtrainSub,ytrainSub)
predictions=gb.predict(xvalidationSub)

print('confusion matrix:')
print(confusion_matrix(yvalidationSub,predictions))
print()
print('confusion report')
print(classification_report(yvalidationSub,predictions))

confusion matrix:
[[121  18]
 [ 24  60]]

confusion report
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       139
           1       0.77      0.71      0.74        84

    accuracy                           0.81       223
   macro avg       0.80      0.79      0.80       223
weighted avg       0.81      0.81      0.81       223



In [23]:
yScoresGB=gb.decision_function(xvalidationSub)
fprGB,tprGB,_=roc_curve(yvalidationSub,yScoresGB)
rocAUCgb=auc(fprGB,tprGB)
print('area under roc curve={:0.2f}'.format(rocAUCgb))

area under roc curve=0.86
